<a href="https://colab.research.google.com/github/fulati/GumGum/blob/fulati/GumGum_Latest__Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!pip install datasets

Tue Oct  1 07:39:22 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.09                 Driver Version: 561.09         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070      WDDM  |   00000000:06:00.0  On |                  N/A |
| 38%   55C    P8             21W /  220W |     621MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
import pandas as pd
import numpy as np
import itertools

from datasets import Dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score

C:\Users\t0pqt\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')


# bbkpi_df = pd.read_csv('/content/drive/MyDrive/gumgum_model_self/gumgum_dataset/bbkpi_gold.csv')


bbkpi_df = pd.read_csv('C:/Users/t0pqt/Desktop/bbkpi_gold.csv')
ground_truth_cols = [c for c in bbkpi_df.columns[2:] if not c.endswith('_pred')]
verity_prediction_cols = [c for c in bbkpi_df.columns[2:] if c.endswith('_pred')]

bbkpi_df.head()

,mp4,text,Arms and Ammunition,Arms and Ammunition_pred,Crime & Harmful acts to individuals and Society and Human Right Violation,Crime & Harmful acts to individuals and Society and Human Right Violation_pred,"Death, Injury or Military Conflict","Death, Injury or Military Conflict_pred",Hate Speech & Acts of aggression,Hate Speech & Acts of aggression_pred,"Obscenity and Profanity, including language, gestures, and explicitly gory, graphic or repulsive content intended to shock and disgust","Obscenity and Profanity, including language, gestures, and explicitly gory, graphic or repulsive content intended to shock and disgust_pred",Illegal Drugs/Tobacco/e-cigarettes/Vaping/Alcohol,Illegal Drugs/Tobacco/e-cigarettes/Vaping/Alcohol_pred
0,a55bf111-6c4b-59df-9d62-ed8d1da6b823_0.mp4,"Okay, let's go. Sarang is excited. Do we have ...",False,False,False,False,False,False,False,False,False,False,False,False
1,7ee68d88-b2f2-5426-9679-a94b3a3afd6c_16.mp4,"And where did you come from? Lukey, do you hav...",True,False,False,False,False,False,False,False,False,False,False,False
2,cc5b03ca-d343-5946-b716-d066aa84a9cd_4.mp4,Come here. This is Mr. Peep. He's very sweet a...,False,False,False,False,False,False,False,False,False,False,False,False
3,de3420b6-6932-5546-ba08-2f5dbc9af725_15.mp4,"to really go forward with this relationship, a...",False,False,False,False,False,False,False,False,False,False,False,False
4,5b7e863b-8fa4-5dbe-919c-3fbe222a657f_59.mp4,"Victoria, Victoria. I knew this would be hard....",False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
# prod_silver_df = pd.read_excel('/content/drive/MyDrive/gumgum_model_self/gumgum_dataset/prod_silver.xlsx')

prod_silver_df = pd.read_excel('C:/Users/t0pqt/Desktop/prod_silver.xlsx')
prod_silver_df = prod_silver_df[['TIMESTAMP', 'VIDEO_UUID', 'INTERVAL_ID', 'TEXT', 'LANGUAGE_CODE'] + ground_truth_cols]
prod_silver_df.head()

,TIMESTAMP,VIDEO_UUID,INTERVAL_ID,TEXT,LANGUAGE_CODE,Arms and Ammunition,Crime & Harmful acts to individuals and Society and Human Right Violation,"Death, Injury or Military Conflict",Hate Speech & Acts of aggression,"Obscenity and Profanity, including language, gestures, and explicitly gory, graphic or repulsive content intended to shock and disgust",Illegal Drugs/Tobacco/e-cigarettes/Vaping/Alcohol
0,2024-08-17 10:24:00,87ed6d3b-d2bf-5a9a-86a4-a3a1715e5a1c,7,How are you doing? It's funny because if you d...,en,False,False,False,False,True,False
1,2024-06-02 05:29:00,eb78f57c-503e-5b0f-bce5-8e346c8aff00,2,I'm chillin' out with the crew in the schoolya...,en,False,False,False,False,False,False
2,2024-02-05 05:53:00,40bbd600-c016-57c2-aaf1-03f33046b1ea,55,"A special dessert for the lady. Chaco, it's to...",en,False,False,False,False,False,False
3,2024-01-30 18:57:00,3ff7a6f5-150b-5d1d-8b57-6464f184f0c2,15,"Take that, too. Oh! Oh, no. Aren't you going ...",en,False,False,False,False,False,False
4,2023-12-21 01:02:00,17a185fd-37a7-531a-be03-82575f8ed4e5,37,Chicken! Chicken! Chicken! Chicken! Chicken! ...,en,False,False,True,False,False,False


In [ ]:
# # Data Sanity Check: Ensure there are no null or empty values in the text
# assert not prod_silver_df['TEXT'].isnull().any(), "Null values found in prof_silver TEXT column"
# assert not (prod_silver_df['TEXT'].str.len() == 0).any(), "Empty text found in prof_silver TEXT column"
# assert not bbkpi_df['text'].isnull().any(), "Null values found in bbkpi_df text column"
# assert not (bbkpi_df['text'].str.len() == 0).any(), "Empty text found in bbkpi_df text column"


# Check data types
# assert prod_silver_df['TEXT'].dtype == 'object', "TEXT column should be strings"
# assert all(prod_silver_df[ground_truth_cols].applymap(np.isreal).all()), "Labels should be numeric"
# assert bbkpi_df['text'].dtype == 'object', "text column in bbkpi_df should be strings"

In [ ]:
print(prod_silver_df.shape)
print(bbkpi_df.shape)

(9602, 11)
(828, 14)


In [ ]:
# Remove rows with NaN or empty text values
prod_silver_df = prod_silver_df.dropna(subset=['TEXT'])  # Drop rows where TEXT is NaN
prod_silver_df = prod_silver_df[prod_silver_df['TEXT'].str.len() > 0]  # Drop rows where TEXT is empty

bbkpi_df = bbkpi_df.dropna(subset=['text'])  # Drop rows where text is NaN
bbkpi_df = bbkpi_df[bbkpi_df['text'].str.len() > 0]  # Drop rows where text is empty

print(prod_silver_df.shape)
print(bbkpi_df.shape)

(9602, 11)
(804, 14)


In [ ]:

pre_train_df = pd.DataFrame({'text' : prod_silver_df['TEXT'].tolist(),
                        'labels' : [x for x in prod_silver_df[ground_truth_cols].values.astype(float)]})
pre_train_df = pre_train_df[pre_train_df['text'].apply(lambda s : isinstance(s, str) and len(s) >= 1)]

print(len(pre_train_df))
dataset = Dataset.from_pandas(pre_train_df)
pre_train_df.head()

9602


,text,labels
0,How are you doing? It's funny because if you d...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
1,I'm chillin' out with the crew in the schoolya...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,"A special dessert for the lady. Chaco, it's to...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,"Take that, too. Oh! Oh, no. Aren't you going ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,Chicken! Chicken! Chicken! Chicken! Chicken! ...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"


In [ ]:
pre_test_df = pd.DataFrame({'text' : bbkpi_df['text'].tolist(),
                        'labels' : [x for x in bbkpi_df[ground_truth_cols].values.astype(float)]})
pre_test_df = pre_test_df[pre_test_df['text'].apply(lambda s : isinstance(s, str) and len(s) >= 1)]

print(len(pre_test_df))
test_dataset = Dataset.from_pandas(pre_test_df)
pre_test_df.head()

804


,text,labels
0,"Okay, let's go. Sarang is excited. Do we have ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"And where did you come from? Lukey, do you hav...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,Come here. This is Mr. Peep. He's very sweet a...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,"to really go forward with this relationship, a...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,"Victoria, Victoria. I knew this would be hard....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [ ]:

# Additional new Models and their respective tokenizers
models = [
    {'model': 'bert-base-uncased', 'tokenizer': 'bert-base-uncased'},
    #{'model': 'roberta-base', 'tokenizer': 'roberta-base'},
    #{'model': 'microsoft/deberta-base', 'tokenizer': 'microsoft/deberta-base'},   #should use 1e-5 learning rate
    #{'model': 'distilbert-base-uncased', 'tokenizer': 'distilbert-base-uncased'},


    #{'model': 'albert-base-v2', 'tokenizer': 'albert-base-v2'},
    #{'model': 'xlnet-base-cased', 'tokenizer': 'xlnet-base-cased'}
]

results = []


# Define hyperparameter values to test
batch_sizes = [16] #8, 16, 32      32, 5, both, all 3, microsoft/deberta-base
epochs = [3] #3, 5 is good
learning_rates = [5e-5]  #1e-5, 5e-5
weight_decays = [0.01] # 0, 0.1, 0.01

In [ ]:
for model_info in models:
    model_name = model_info['model']
    tokenizer_name = model_info['tokenizer']
    print(f"Training with {model_name} and {tokenizer_name}")

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    def tokenize_function(examples):
      return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_train_dataset = dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)
    all_datasets = tokenized_train_dataset.train_test_split(test_size=0.2)
    train_dataset = all_datasets["train"]
    val_dataset = all_datasets["test"]


    id2label = dict((i, l) for i, l in enumerate(ground_truth_cols))
    label2id = dict((l, i) for i, l in id2label.items())

    # Loop over hyperparameter combinations
    for batch_size, num_epochs, learning_rate, weight_decay in itertools.product(batch_sizes, epochs, learning_rates, weight_decays):
        print(f"\nHyperparameters: batch_size={batch_size}, epochs={num_epochs}, learning_rate={learning_rate}, weight_decay={weight_decay}")

        # Initialize the model for each hyperparameter combination
        model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                  problem_type="multi_label_classification",
                                                                  num_labels=len(id2label),
                                                                  id2label=id2label,
                                                                  label2id=label2id).to('cuda')

        training_args = TrainingArguments(
            output_dir="./results",
            eval_strategy="epoch",
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            logging_dir='./logs',
        )

        def compute_multilabel_metrics(predictions, labels, threshold=0.5):
            # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
            sigmoid = torch.nn.Sigmoid()
            probs = sigmoid(torch.Tensor(predictions))
            y_pred = np.zeros(probs.shape)
            # next, use threshold to turn them into integer predictions
            y_pred[np.where(probs >= threshold)] = 1

            # finally, compute metrics
            y_true = labels
            f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
            f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
            accuracy = accuracy_score(y_true, y_pred)
            # return as dictionary
            metrics = {'f1_micro': f1_micro_average,
                      'f1_macro': f1_macro_average,
                      'accuracy': accuracy}
            return metrics

        def compute_metrics(p):
            preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions

            result = compute_multilabel_metrics(
                predictions=preds,
                labels=p.label_ids)
            return result


        class MultilabelTrainer(Trainer):
            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get("logits")

                logits = logits.contiguous()

                loss_fct = torch.nn.BCEWithLogitsLoss()
                loss = loss_fct(logits, labels.float())
                return (loss, outputs) if return_outputs else loss

        print(f"Training with {model_name}...")

        trainer = MultilabelTrainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics
        )

        trainer.train()

        #Evaluate and store results
        metrics = trainer.evaluate()

        results.append({
                'model': model_name,
                'batch_size': batch_size,
                'epochs': num_epochs,
                'learning_rate': learning_rate,
                'weight_decay': weight_decay,
                'eval_accuracy': metrics['eval_accuracy'],
                'eval_f1_macro': metrics['eval_f1_macro'],
                'eval_f1_micro': metrics['eval_f1_micro'],
                'eval_loss': metrics['eval_loss'],
            })


Training with bert-base-uncased and bert-base-uncased


C:\Users\t0pqt\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████████████████████████████████████████████████████████████| 804/804 [00:00<00:00, 4562.20 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Hyperparameters: batch_size=16, epochs=3, learning_rate=5e-05, weight_decay=0.01
Training with bert-base-uncased...


C:\Users\t0pqt\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy
1,No log,0.129469,0.598736,0.321138,0.795940
2,0.182300,0.101631,0.719495,0.549492,0.832900
3,0.108500,0.100240,0.732044,0.614431,0.836543


In [ ]:
# Print and compare results
print("\nAll Results:")
for result in results:
    print(f"Model: {result['model']}, Batch Size: {result['batch_size']}, Epochs: {result['epochs']}, "
          f"Learning Rate: {result['learning_rate']}, Weight Decay: {result['weight_decay']}, "
          f"F1 Macro: {result['eval_f1_macro']}, Accuracy: {result['eval_accuracy']}")


All Results:
Model: bert-base-uncased, Batch Size: 16, Epochs: 3, Learning Rate: 5e-05, Weight Decay: 0.01, F1 Macro: 0.6144311478323309, Accuracy: 0.8365434669442998


In [ ]:
# Optionally, save the results to a CSV file
df_results = pd.DataFrame(results)
df_results.to_csv("hyperparameter_results_learningrate_model.csv", index=False)

In [ ]:
torch.cuda.empty_cache()